In [1]:
#import dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
#use the citipy module to determine city based on latitude and longitude
from citipy import citipy
#import the datetime module from the datetime library
from datetime import datetime
#for API requests
import requests
#import the API key
from config import weather_api_key
# Import the time module.
import time
# Import linear regression from the SciPy stats module
from scipy.stats import linregress
import csv
import os
import gmaps


In [2]:
#file to load
weather_data_to_load ="data/WeatherPy_challenge.csv"


In [3]:
#read csv files into a df
weather_data_df=pd.read_csv(weather_data_to_load)
weather_data_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current description,Rain (last 3 hrs),Snow (last 3 hrs)
0,0,Port Moresby,PG,2020-04-19 19:03:54,-9.44,147.18,78.35,83,51,5.59,broken clouds,0,0
1,1,Hobart,AU,2020-04-19 18:58:22,-42.88,147.33,51.80,76,100,5.82,overcast clouds,0,0
2,2,Moyale,KE,2020-04-19 19:03:56,3.52,39.06,68.58,68,100,3.78,light rain,0,0
3,3,Broken Hill,AU,2020-04-19 19:03:57,-31.95,141.43,69.80,28,75,18.34,broken clouds,0,0
4,4,Rantepao,ID,2020-04-19 19:03:57,-2.97,119.90,66.38,73,89,2.19,light rain,0,0
5,5,Hofn,IS,2020-04-19 19:02:17,64.25,-15.21,44.42,85,100,7.54,overcast clouds,0,0
6,6,Mataura,NZ,2020-04-19 19:02:17,-46.19,168.86,44.01,80,93,7.00,light rain,0,0
7,7,Clyde River,CA,2020-04-19 19:03:59,70.47,-68.59,6.80,65,20,19.46,few clouds,0,0
8,8,San Quintin,MX,2020-04-19 19:04:00,30.48,-115.95,63.99,63,99,15.23,overcast clouds,0,0
9,9,Ancud,CL,2020-04-19 19:02:25,-41.87,-73.82,53.60,81,75,12.75,broken clouds,0,0


In [4]:
weather_data_df.dtypes

City_ID                  int64
City                    object
Country                 object
Date                    object
Lat                    float64
Lng                    float64
Max Temp               float64
Humidity                 int64
Cloudiness               int64
Wind Speed             float64
Current description     object
Rain (last 3 hrs)        int64
Snow (last 3 hrs)        int64
dtype: object

In [5]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
rain_req = str(input("Do you want it to be raining? (yes/no) "))
snow_req = str(input("Do you want it to be snowing? (yes/no) "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 95
Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) no


In [6]:
# Filter the dataset to find the cities that fit the criteria.
#filter if rain_req  
if rain_req == "yes":
    preferred_weather_df1 = weather_data_df.loc[(weather_data_df["Max Temp"] <= max_temp) & (weather_data_df["Max Temp"] >= min_temp) & (weather_data_df["Rain (last 3 hrs)"]>0)  ]
else:
    preferred_weather_df1 = weather_data_df.loc[(weather_data_df["Max Temp"] <= max_temp) & (weather_data_df["Max Temp"] >= min_temp)   ]
#filter if snow_req
if snow_req == "yes":
    preferred_weather_df2 = preferred_weather_df1.loc[(preferred_weather_df1["Max Temp"] <= max_temp) & (preferred_weather_df1["Max Temp"] >= min_temp) & (preferred_weather_df1["Snow (last 3 hrs)"]>0)  ]
else:
    preferred_weather_df2 = preferred_weather_df1.loc[(preferred_weather_df1["Max Temp"] <= max_temp) & (preferred_weather_df1["Max Temp"] >= min_temp)   ]

preferred_weather_df2

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current description,Rain (last 3 hrs),Snow (last 3 hrs)
0,0,Port Moresby,PG,2020-04-19 19:03:54,-9.44,147.18,78.35,83,51,5.59,broken clouds,0,0
10,10,Bengkulu,ID,2020-04-19 18:58:06,-3.80,102.27,78.82,81,100,3.74,overcast clouds,0,0
13,13,Rikitea,PF,2020-04-19 19:00:03,-23.12,-134.97,76.75,70,59,7.87,broken clouds,0,0
18,18,Vaini,TO,2020-04-19 19:04:09,-21.20,-175.20,77.00,83,40,9.17,scattered clouds,0,0
22,22,Sulangan,PH,2020-04-19 19:04:13,11.14,123.72,82.18,81,83,12.48,light rain,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
540,540,Tagusao,PH,2020-04-19 19:09:30,9.19,117.81,81.14,77,19,3.22,few clouds,0,0
546,546,Chabahar,IR,2020-04-19 19:09:32,25.29,60.64,80.96,69,55,6.98,broken clouds,0,0
549,549,Manama,BH,2020-04-19 19:05:44,26.22,50.58,78.80,78,100,9.17,light rain,0,0
555,555,Phanat Nikhom,TH,2020-04-19 19:09:36,13.45,101.18,87.01,62,36,7.05,scattered clouds,0,0


In [7]:
#create the output file (CSV)
output_data_file = "data/WeatherPy_vacation.csv"
#export the challenge weather data into a CSV
preferred_weather_df2.to_csv(output_data_file, index_label="City_ID")

In [8]:
import gmaps
# Import the API key.
from config import g_key
gmaps.configure(api_key=g_key)

#add heatmap hotel, city, country, desc
locations = preferred_weather_df2[["Lat", "Lng"]]
#max_temp = preferred_weather_df2["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
#heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
#fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
#call the figure to plot data
fig


Figure(layout=FigureLayout(height='420px'))

In [9]:
#create dataframe hotel_df to store hotel names along with city, country, max temp, and coordinates
hotel_df = preferred_weather_df2[["City", "Current description","Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Current description,Country,Max Temp,Lat,Lng,Hotel Name
0,Port Moresby,broken clouds,PG,78.35,-9.44,147.18,
10,Bengkulu,overcast clouds,ID,78.82,-3.80,102.27,
13,Rikitea,broken clouds,PF,76.75,-23.12,-134.97,
18,Vaini,scattered clouds,TO,77.00,-21.20,-175.20,
22,Sulangan,light rain,PH,82.18,11.14,123.72,
...,...,...,...,...,...,...,...
540,Tagusao,few clouds,PH,81.14,9.19,117.81,
546,Chabahar,broken clouds,IR,80.96,25.29,60.64,
549,Manama,light rain,BH,78.80,26.22,50.58,
555,Phanat Nikhom,scattered clouds,TH,87.01,13.45,101.18,


In [10]:
# Set the parameters to search for a hotel in Paris.
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key}

#iterate through dataframe
for index, row in hotel_df.iterrows():
    #get latitude and longitude
    lat = row["Lat"]
    lng = row["Lng"]

    #add the latitude and longitude to location key for the params dictionary in gmaps
    params["location"] = f"{lat},{lng}"

    
     # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except(IndexError):
        print("Hotel not found... skipping.")
    
    

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [11]:
hotel_df

,City,Current description,Country,Max Temp,Lat,Lng,Hotel Name
0,Port Moresby,broken clouds,PG,78.35,-9.44,147.18,Lamana Hotel
10,Bengkulu,overcast clouds,ID,78.82,-3.80,102.27,Grage Hotel Bengkulu
13,Rikitea,broken clouds,PF,76.75,-23.12,-134.97,Pension Maro'i
18,Vaini,scattered clouds,TO,77.00,-21.20,-175.20,Keleti Beach Resort
22,Sulangan,light rain,PH,82.18,11.14,123.72,
...,...,...,...,...,...,...,...
540,Tagusao,few clouds,PH,81.14,9.19,117.81,"Shaman's Breath- Beach, Bed & Breakfast"
546,Chabahar,broken clouds,IR,80.96,25.29,60.64,هتل گدروشیا
549,Manama,light rain,BH,78.80,26.22,50.58,Al Safir Hotel & Tower
555,Phanat Nikhom,scattered clouds,TH,87.01,13.45,101.18,I-Destiny Gallery Resort


In [12]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current description</dt><dd>{Current description} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [15]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))